# QFin Trading Team Workshop 4
## Creating an Algorithm

In [120]:
!pip install qfinuwa
# update the package
!pip install --upgrade qfinuwa

In [121]:
from qfinuwa import Indicators
import pandas as pd

### But first - some bug fixes

- indicator are 1 billion times faster to test
- no need to pip install requests
- documentation

In [122]:

import numpy as np

class CustomIndicators(Indicators):

    @Indicators.MultiIndicator
    def breakout_strategy(self, stock, ORB_WINDOW=20, ORB_THRESHOLD=0.05):
        # Initialize ORB values
        prev_date = None

        # Calculate ORB breakout
        breakout_signal = pd.Series(0, index=stock.index)
        for i in range(len(stock)):
            # If new date, start new ORB
            curr_date = stock['time'][i].split(" ")[0]
            if curr_date != prev_date:
                prev_date = curr_date
                max_orb = stock['high'][i:i+ORB_WINDOW].max()
                min_orb = stock['low'][i:i+ORB_WINDOW].min()

            # print(f"Close: {stock['close'][i]}, Max: {max_orb}, Min: {min_orb}")
            # Calculate breakout signal
            # prints the price difference between the current close price and the max orb

            if stock['close'][i] > max_orb * (1 + ORB_THRESHOLD):
                breakout_signal[i] = 1
            elif stock['close'][i] < min_orb * (1 - ORB_THRESHOLD):
                breakout_signal[i] = -1
        
        return {
            "breakout_signal": breakout_signal,
            "max_orb": max_orb,
            "min_orb": min_orb
        }


    @Indicators.MultiIndicator
    def rsi(self, stock, RSI_WINDOW=14, RSI_UPPER=70, RSI_LOWER=30):
        # Calculate RSI
        delta = stock['close'].diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(RSI_WINDOW).mean()
        avg_loss = loss.rolling(RSI_WINDOW).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))

        # Determine buy and sell signals
        buy_signal = rsi > RSI_UPPER
        sell_signal = rsi < RSI_LOWER

        return {
            "rsi": rsi,
            "buy_signal": buy_signal,
            "sell_signal": sell_signal
        }


In [123]:
df1 = pd.read_csv('data/AAPL.csv')
df2 = pd.read_csv('data/GOOG.csv')

ind = CustomIndicators()

# output = ind.bollinger_bands(df1)
output = ind.breakout_strategy(df1)

In [124]:
# prints values of output that are not 0
# for value in output['breakout_signal']:
# counts the 1,0,-1 values in the breakout_signal column
output['breakout_signal'].value_counts()

 0    429231
 1      1181
-1      1018
dtype: int64

## Strategy Class

Similar to the ``Indicators`` class, we can implement a strategy by extending ``qfinuwa.Strategy``.

The null strategy would be implemented as follows:

In [125]:
from qfinuwa import Strategy

class MyStrategy(Strategy):
    
    def __init__(self):
        return
    
    def on_data(self, data, indicators, portfolio):
       return
    
    def on_finish(self):
        return

The strategy offers custom implementation of three functions.

- ``__init``: called on creation of the strategy
- ``on_data``: called every data tick 
- ``on_finish``: return an object to be appended to the results

### ``__init__``

You can create variables here that can be accessed by ``on_data`` at runtime. 

Similar to each of your indicator functions, ``kwargs`` can be specified as hyperparamters (more on this later).

### ``on_data``

``on_data`` expects 3 additional arugments that will be populated at runtime.

- ``data``: the historic data up to this point
- ``indicators``: the historic value of your custom indicators
- ``portfolio``: an object that manages your position

### ``on_finish``

Should return an object that will be added to the results. An example use case is if you want to measure some statistic over time, and the backtester doesn't offer it, so you record it yourself during runtime and return it via this function.

### Example


In [126]:
class MyStrategy(Strategy):
    
    def __init__(self, quantity=5):
        self.quantity = quantity
    
    def on_data(self, prices, indicators, portfolio):

        print(indicators)
        # If current price is below lower Bollinger Band, enter a long position
        # if indicators['rsi']['AAPL'][-1] == 1 and breakout_indicators['breakout_signal']['AAPL'][-1] == 1:
        #     # Exit short position and enter long position
        #     portfolio.exit_position('short', 'AAPL', quantity=self.quantity)
        #     portfolio.enter_position('long', 'AAPL', quantity=self.quantity)

        
        # # If current price is above upper Bollinger Band, enter a short position
        # if(prices['close']['AAPL'][-1] > indicators['upper_bollinger']['AAPL'][-1]):
        #     portfolio.exit_position('short', 'AAPL', quantity=self.quantity)
        #     portfolio.enter_position('long', 'AAPL', quantity=self.quantity)

    def on_finish(self):
        return 'Hello there!'

## Portfolio Class

To execute a trade, use one of the following function on the ``portfolio`` argument.

```py
portfolio.enter_position('long', 'AAPL', quantity=100)
portfolio.exit_position('short', 'GOOG', quantity=50)
portfolio.enter_position('long', 'TSLA', quantity=20)
portfolio.exit_position('short', 'AAPL', quantity=1)
```

``enter_position`` and ``exit_postion`` will return 0 if successful, 1 if unsucessful and 2 if there was a code error.

Some useful variables:

```py
portfolio.cash
portfolio.holdings
portfolio.stocks
```

## The Backtester

In [127]:
from qfinuwa import Backtester

In [133]:
backtester = Backtester(MyStrategy, CustomIndicators, ['AAPL'], data_folder="./data/"
                        ,days='all', fee=0.002,)

> Precompiling data: 100%|██████████| 431430/431430 [00:05<00:00, 74892.95it/s]


KeyError: 'time'

The backtester may take a few seconds to create itself, but once it is created you shouldn't do it again unless you want to change the data. It takes a long time to create itself because it is precompiling the data for iterating.

### Changing Backtesting Parameters

In [134]:
backtester

NameError: name 'backtester' is not defined

In [ ]:
backtester.fee = 0.02
backtester.starting_balance = 100000
backtester.days = 90

backtester

Backtester:
- Strategy: MyStrategy
	- Params: {'quantity': 5}
- Indicators: CustomIndicators
	- Params: {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 100}, 'etf': {'apple': 0.2, 'goog': 0.3}}
	- SingleIndicators: ['etf']
	- MultiIndicators: ['lower_bollinger', 'upper_bollinger']
- Stocks: ['AAPL', 'GOOG']
- Fee 0.02
- Starting Balance: 10000
- Days: 90

## Changing Strategy Parameters

In [ ]:
print('current params', backtester.strategy.params)
print('default params', backtester.strategy.defaults)

current params {'quantity': 5}
default params {'quantity': 5}


In [ ]:
backtester.strategy.update_params({'quantity': 10})

In [ ]:
backtester.strategy = MyStrategy

## Changing Indicator Parameters

In [ ]:
print('current params', backtester.indicators.params)
print('default params', backtester.indicators.defaults)

current params {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 100}, 'etf': {'apple': 0.2, 'goog': 0.3}}
default params {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 100}, 'etf': {'apple': 0.2, 'goog': 0.3}}


In [ ]:
backtester.indicators.update_params({'etf': {'apple': 0.0}, 'bollinger_bands': {'BOLLINGER_WIDTH': 1, 'WINDOW_SIZE': 50}})

In [ ]:
backtester.indicators = CustomIndicators

## Running Backtests

To run a backtest, use ``Backtester.run``.

``run`` will pick ``cv`` random periods of ``backtester.days`` days and run your strategy on that window of data. If you want reproducibility, use ``seed`` to use the same random generation every time. 

In [ ]:
output = backtester.run(cv=5, seed=2023)

> Running backtest over 5 samples of 90 days: 100%|██████████| 5/5 [00:03<00:00,  1.38it/s]


In [ ]:
output
# output[3]

# output.roi
# output[3].roi

# [run.roi for run in output]


{'strategy': {'quantity': 10}, 'indicator': {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 100}, 'etf': {'apple': 0.2, 'goog': 0.3}}}

Mean ROI:	-0.4134143129212961
STD ROI:	0.38211042007780277

01/10/2022 -> 05/13/2022:	-0.857
08/25/2022 -> 12/28/2022:	-0.875
02/23/2022 -> 06/28/2022:	-0.194
01/13/2022 -> 05/18/2022:	-0.204
11/24/2021 -> 03/29/2022:	0.064

AVERAGED RESULTS FOR 5 RUNS:
|                 |     AAPL |   GOOG |      Net |
|-----------------|----------|--------|----------|
| n_trades        |    220.6 |      0 |    220.6 |
| mean_per_trades | -289.063 |      0 | -144.531 |
| std_trades      |  9.01043 |      0 |  9.01043 |
| n_longs         |    110.6 |      0 |    110.6 |
| mean_per_longs  | -578.558 |      0 | -289.279 |
| std_longs       |    5.119 |      0 |    5.119 |
| n_shorts        |      110 |      0 |      110 |
| mean_per_shorts |  0.43198 |      0 |  0.21599 |
| std_shorts      |  6.68324 |      0 |  6.68324 |

You can either use ``Backtester.strategy.update_params`` to update parameters, or just give them to ``run``.

In [ ]:
S = {
    'quantity': 90
}

I = {
    'bollinger_bands': {
        'BOLLINGER_WIDTH': 1,
        'WINDOW_SIZE': 50
    },
    'etf': {
        'apple': 0.0
    }
}


output = backtester.run(strategy_params = S, indicator_params = I, cv=5)

> Running backtest over 5 samples of 90 days: 100%|██████████| 5/5 [00:03<00:00,  1.38it/s]


## Hyper-Parameter Sweep

To run a basic grid-search over all your hyperparameters, use ``Backtester.run_grid_search``.

In [ ]:
output = backtester.run_grid_search(
    strategy_params={'quantity': [1, 10]},
    indicator_params={'bollinger_bands': {
                            'BOLLINGER_WIDTH': [1, 2], 
                            }, 
                        'etf': {'apple': [0.0, 0.5]}},
    cv=2)

> Backtesting the across the following ranges:
Agorithm Parameters {'quantity': [1, 10]}
Indicator Parameters {'bollinger_bands': {'BOLLINGER_WIDTH': [1, 2], 'WINDOW_SIZE': 100}, 'etf': {'apple': [0.0, 0.5], 'goog': 0.3}}


Running paramter sweep (cv=2): 100%|██████████| 8/8 [00:20<00:00,  2.51s/it]


In [ ]:
output
output[0]


{'strategy': {'quantity': 1}, 'indicator': {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 100}, 'etf': {'apple': 0.0, 'goog': 0.3}}}

Mean ROI:	-0.7168047709784728
STD ROI:	0.22559358002675983

04/29/2022 -> 09/01/2022:	-0.942
12/07/2021 -> 04/11/2022:	-0.491

AVERAGED RESULTS FOR 2 RUNS:
|                 |      AAPL |   GOOG |       Net |
|-----------------|-----------|--------|-----------|
| n_trades        |      3587 |      0 |      3587 |
| mean_per_trades |  -2.59445 |      0 |  -1.29723 |
| std_trades      |   3.35652 |      0 |   3.35652 |
| n_longs         |    1807.5 |      0 |    1807.5 |
| mean_per_longs  |  -4.32092 |      0 |  -2.16046 |
| std_longs       |    3.4156 |      0 |    3.4156 |
| n_shorts        |    1779.5 |      0 |    1779.5 |
| mean_per_shorts | -0.867991 |      0 | -0.433996 |
| std_shorts      |   1.89827 |      0 |   1.89827 |

## Plotting

In [ ]:
from qfinuwa import Plotting

In [ ]:
multirun = output[0]
singlerun = multirun[0]

Plotting.plot_result(singlerun, stocks=['AAPL'], transactions_on='AAPL')